In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, Normalizer
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
import joblib

In [3]:
data_property_cell = pd.read_csv("../data/qmof_property_cell.csv", index_col=1) # index_col=1 id "filename"
data_property_cell = data_property_cell.drop("Unnamed: 0", axis=1) # drop numbers of rows
property_cell_cols = ["pld", "lcd", "a", "b", "c", "alpha", "betta", "gamma", "volume", "spacegroupNumber"]
data_property_cell = data_property_cell.loc[:,property_cell_cols].rename({"betta": "beta"}, axis=1)

In [4]:
data_linker_mordred = pd.read_csv("../data/mordred_descriptors.csv", index_col=0)

In [5]:
p = len(data_linker_mordred.values[np.isnan(data_linker_mordred)]) / len(data_linker_mordred) / len(data_linker_mordred.iloc[0])
print(f"Mordred data has {p*100:.2n} % of nan")

Mordred data has 25 % of nan


In [6]:
data_linker_mordred.shape

(7109, 1826)

Load node data

In [7]:
data_node = pd.read_csv("../data/node_descriptors.csv", index_col=0)
data_node = data_node.loc[data_node["n_types_metals"] == 1.]
data_node = data_node.loc[:, ['n_metals', 'Atomic_Number', 'Atomic_Weight',
                              'Atomic Radius', 'Mulliken EN', 'polarizability(A^3)', 'electron affinity(kJ/mol)']]


In [8]:
data_qmof = pd.concat([data_property_cell, data_linker_mordred, data_node], axis=1)

In [9]:
data_qmof.shape

(7463, 1843)

In [10]:
data_qmof = data_qmof.loc[list({*list(data_linker_mordred.index)} & {*list(data_node.index)})]

In [11]:
data_qmof.shape

(6797, 1843)

In [12]:
data_qmof.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6797 entries, boydwoo_str_m5_Al_o4_o17_sra_sym_74 to WADDOX_FSR
Columns: 1843 entries, pld to electron affinity(kJ/mol)
dtypes: float64(1842), int64(1)
memory usage: 95.6+ MB


In [13]:
train, test = train_test_split(data_qmof, test_size=0.2, random_state=0)

In [14]:
train.shape

(5437, 1843)

In [15]:
len(train["beta"].dropna())

5437

In [16]:
train["pld"].std()

4.303728942107217

In [17]:
for col in train.columns:
    print(f"{col} has {((5437 - len(train[col].dropna()))/5437)*100:.0f} % of nan and {train[col].std():.4f} std")

pld has 0 % of nan and 4.3037 std
lcd has 0 % of nan and 5.1598 std
a has 0 % of nan and 4.8369 std
b has 0 % of nan and 4.8338 std
c has 0 % of nan and 6.0162 std
alpha has 0 % of nan and 14.2155 std
beta has 0 % of nan and 12.2899 std
gamma has 0 % of nan and 14.3865 std
volume has 0 % of nan and 3412.0627 std
spacegroupNumber has 0 % of nan and 43.0058 std
ABC has 0 % of nan and 6.7819 std
ABCGG has 0 % of nan and 4.9480 std
nAcid has 0 % of nan and 1.0135 std
nBase has 0 % of nan and 0.6974 std
SpAbs_A has 0 % of nan and 10.9278 std
SpMax_A has 0 % of nan and 0.2252 std
SpDiam_A has 0 % of nan and 0.4553 std
SpAD_A has 0 % of nan and 10.9278 std
SpMAD_A has 0 % of nan and 0.0786 std
LogEE_A has 0 % of nan and 0.5081 std
VE1_A has 0 % of nan and 0.7094 std
VE2_A has 0 % of nan and 0.0775 std
VE3_A has 0 % of nan and 0.6847 std
VR1_A has 0 % of nan and 303.1886 std
VR2_A has 0 % of nan and 7.1422 std
VR3_A has 0 % of nan and 1.4251 std
nAromAtom has 0 % of nan and 6.1421 std
nAromBon

In [18]:
from pteproc_model import PreprocessingModel

In [19]:
# p_dorp means we drop columns with more than 5% of nan values
model = PreprocessingModel(p_drop=0.9, threshold=1e-3, normalizer="minmax")

In [20]:
transformed_train = model.fit_transform(train)

In [21]:
transformed_train.shape

(5437, 1169)

In [22]:
pd.options.display.max_columns = 100
transformed_train

,pld,lcd,a,b,c,alpha,beta,gamma,volume,spacegroupNumber,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,VE1_A,VE2_A,VE3_A,VR3_A,nAromAtom,nAromBond,nAtom,nHeavyAtom,nHetero,nH,nC,nN,nO,nS,nP,nF,nCl,nBr,nI,nX,ATS0dv,ATS1dv,ATS2dv,ATS3dv,ATS4dv,ATS5dv,ATS6dv,ATS7dv,ATS8dv,ATS0d,...,JGI2,JGI3,JGI4,JGI5,JGI6,JGI7,JGI8,JGI9,JGI10,JGT10,Diameter,Radius,TopoShapeIndex,PetitjeanIndex,Vabc,VAdjMat,MWC01,MWC02,MWC03,MWC04,MWC05,MWC06,MWC07,MWC08,MWC09,MWC10,TMWC10,SRW02,SRW04,SRW05,SRW06,SRW07,SRW08,SRW09,SRW10,TSRW10,MW,AMW,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2,n_metals,Atomic_Number,Atomic_Weight,Atomic Radius,Mulliken EN,polarizability(A^3),electron affinity(kJ/mol)
boydwoo_str_m3_o15_o17_nbo_sym_15,0.270155,0.448419,0.562968,0.535403,0.327225,0.598806,0.947294,0.807056,0.151355,0.424779,0.215447,0.285505,0.200000,0.000000,0.192259,0.671013,0.671013,0.192259,0.753604,0.641229,0.379500,0.173225,0.615106,0.617473,0.192308,0.194444,0.162651,0.207547,0.162162,0.073529,0.177083,0.000000,0.153846,0.0,0.000000,0.0,0.0,0.000,0.5,0.150,0.207422,0.204662,0.232709,0.219959,0.165259,0.123769,0.073609,0.043917,0.022705,0.204748,...,0.383632,0.322790,0.372181,0.282140,0.430710,0.379780,0.301263,0.153846,0.333333,0.513409,8.000000e-08,4.500000e-08,0.616667,0.740260,0.138720,0.676692,0.204000,0.714934,0.759136,0.795275,0.822206,0.823053,0.818648,0.815094,0.811957,0.809606,0.371058,0.650781,0.739083,0.000000,0.803791,0.000000,0.805624,0.000000,0.796101,0.319821,0.280441,0.619536,0.256345,0.216612,0.223772,0.226116,0.186717,0.090909,0.303371,0.252930,0.133333,0.841924,0.018450,0.014184
boydwoo_str_m3_o3_o5_pcu_sym_50,0.205213,0.234178,0.179022,0.419676,0.271395,0.571950,0.530553,0.443976,0.063217,0.000000,0.217368,0.271941,0.133333,0.000000,0.209805,0.553953,0.553953,0.209805,0.766404,0.575269,0.333386,0.237499,0.555415,0.573176,0.256410,0.222222,0.216867,0.213836,0.063063,0.196078,0.222222,0.027778,0.102564,0.0,0.000000,0.0,0.0,0.000,0.0,0.000,0.191967,0.195880,0.200924,0.174566,0.135913,0.136482,0.127179,0.112580,0.106866,0.205737,...,0.333829,0.192347,0.288039,0.132273,0.200624,0.126198,0.084601,0.159512,0.162728,0.421721,9.333333e-08,4.666667e-08,0.791667,0.880808,0.206496,0.615530,0.208000,0.640554,0.672639,0.702646,0.722845,0.722872,0.717382,0.714024,0.710540,0.708399,0.348282,0.587563,0.665203,0.000000,0.720417,0.000000,0.717770,0.000000,0.705399,0.304126,0.131982,0.157498,0.197970,0.204126,0.196990,0.160040,0.203425,0.090909,0.303371,0.252930,0.133333,0.841924,0.018450,0.014184
ARUBIA_FSR,0.036872,0.036091,0.196981,0.203636,0.144698,0.571383,0.331739,0.172511,0.024125,0.004425,0.187235,0.226357,0.100000,0.000000,0.175257,0.550664,0.550664,0.175257,0.760848,0.579817,0.351214,0.226454,0.565600,0.568936,0.192308,0.166667,0.183735,0.183962,0.121622,0.161765,0.166667,0.083333,0.134615,0.0,0.000000,0.0,0.0,0.000,0.0,0.000,0.185588,0.179798,0.177637,0.143361,0.094494,0.073501,0.069547,0.037078,0.037079,0.172107,...,0.372794,0.208642,0.290811,0.325209,0.072963,0.182769,0.106324,0.122967,0.122920,0.471225,1.200000e-07,5.500000e-08,1.000000,1.000000,0.172602,0.621476,0.176000,0.649117,0.684820,0.714509,0.736154,0.734898,0.729383,0.724923,0.721066,0.718120,0.328327,0.592562,0.674345,0.000000,0.733029,0.000000,0.731816,0.000000,0.719669,0.287566,0.116635,0.133191,0.149746,0.171010,0.159363,0.150367,0.171679,0.000000,0.269663,0.224989,0.188889,0.790378,0.038745,0.190071
BAXXAA_FSR,0.029944,0.047473,0.142598,0.173669,0.093549,0.808817,0.536711,0.462336,0.013987,0.004425,0.100270,0.149419,0.200000,0.000000,0.084711,0.536547,0.536547,0.084711,0.661951,0.464176,0.237214,0.312181,0.447613,0.448396,0.076923,0.066667,0.084337,0.094340,0.081081,0.058824,0.083333,0.000000,0.153846,0.0,0.000000,0.0,0.0,0.000,0.0,0.000,0.134554,0.107698,0.116243,0.080000,0.046627,0.025418,0.021242,0.031021,0.000000,0.086053,...,0.529412,0.256790,0.305882,0.590909,0.666667,0.000000,0.000000,0.000000,0.000000,0.624063,6.000000e-08,3.000000e-08,0.750000,0.857143,0

In [23]:
for col in transformed_train.columns:
    print(f"{col} has {((5437 - len(train[col].dropna()))/5437)*100:.0f} % of nan, {transformed_train[col].std():.4f} std, number of unique: {(len(np.unique(transformed_train[col]))):.0f}")

pld has 0 % of nan, 0.1458 std, number of unique: 5408
lcd has 0 % of nan, 0.1610 std, number of unique: 5413
a has 0 % of nan, 0.1466 std, number of unique: 5437
b has 0 % of nan, 0.1513 std, number of unique: 5437
c has 0 % of nan, 0.1283 std, number of unique: 5436
alpha has 0 % of nan, 0.1525 std, number of unique: 5351
beta has 0 % of nan, 0.1788 std, number of unique: 5337
gamma has 0 % of nan, 0.1819 std, number of unique: 5365
volume has 0 % of nan, 0.0696 std, number of unique: 5417
spacegroupNumber has 0 % of nan, 0.1903 std, number of unique: 136
ABC has 0 % of nan, 0.0769 std, number of unique: 2763
ABCGG has 0 % of nan, 0.0902 std, number of unique: 3242
nAcid has 0 % of nan, 0.1014 std, number of unique: 32
nBase has 0 % of nan, 0.0475 std, number of unique: 28
SpAbs_A has 0 % of nan, 0.0747 std, number of unique: 3268
SpMax_A has 0 % of nan, 0.0944 std, number of unique: 3163
SpDiam_A has 0 % of nan, 0.0954 std, number of unique: 3173
SpAD_A has 0 % of nan, 0.0747 std, n

In [24]:
transformed_test = model.transform(test)

In [25]:
transformed_train.shape, transformed_test.shape

((5437, 1169), (1360, 1169))

In [26]:
transformed_train.to_csv("../qmof_datasets/train.csv")
transformed_test.to_csv("../qmof_datasets/test.csv")
joblib.dump(model, "../qmof_datasets/scaler.pkl")

['../qmof_datasets/scaler.pkl']